In [1]:
%gui qt6

# Qt6 stuff
from PyQt6.QtWidgets import QApplication
from PyQt6 import QtWidgets
from PyQt6 import QtGui
from PyQt6 import QtCore
from PyQt6.QtCore import pyqtSignal

In [2]:
# Plotting stuff
import pyqtgraph as pg
import matplotlib.pyplot as plt
import cmasher as cmr

# Science stuff
import numpy as np
import pandas as pd
from spectres import spectres

# Astropy stuff
from astropy.io import fits
from astropy.table import Table
import astropy.units as u
from astropy.units.quantity import Quantity
from astropy.io.ascii import read as ascii_read

# General stuff
import logging
import sys
from pathlib import Path
from itertools import cycle

In [3]:
# zHunter stuff
import zhunter.initialize as init
from zhunter.initialize import DIRS
from zhunter import io
from zhunter.misc import set_up_linked_vb, get_vb_containing, add_crosshair, generate_fake_1D_spectrum
from zhunter.colors import get_gradient
from zhunter.spectroscopic_system import SpecSystemModel, SpecSystem
from zhunter.MainGraphicsWidget import MainGraphicsWidget
from zhunter.data_handler import DataHandler
from zhunter.conversions import convert_flux_to_value, convert_wvlg_to_value, convert_to_bins


In [4]:
# astropalmerio stuff
import astropalmerio.spectra as sp
import astropalmerio.galaxies as gal
import astropalmerio.mc as mc
from astropalmerio.spectra import ergscm2AA
from astropalmerio.spectra import EmissionLine

In [5]:
from zhunter.colors import COLORS
color_style = 'kraken17'
colors = COLORS[color_style]

In [6]:
log = logging.getLogger(__name__)
logging.basicConfig(stream=sys.stdout, level=logging.DEBUG,
    format="%(asctime)s.%(msecs)03d | %(levelname)-8s | %(funcName)s - %(filename)s:%(lineno)d : %(message)s",

                   )
logging.getLogger("matplotlib").setLevel(logging.WARNING)
logging.getLogger("PIL").setLevel(logging.WARNING)

In [7]:
from zhunter.spectrum import OneDSpectrum

# Create Spectrum Object

In [8]:
spec = OneDSpectrum()
# spec = OneDSpectrum(color=colors['specsys'][3])
# spec = OneDSpectrum(color='cyan', width=0.8)

## Load data from a file

In [9]:
# fname = '/Users/palmerio/Code_projects/zHunter/dev/data/test_input_files/GRB050730_UVES_1D.txt.gz'

# spec.load_from_file(fname)

# can also load data directly but data must be quantity
wvlg, flux, unc = generate_fake_1D_spectrum(
    flux_scale=1,
)

spec.load_from_data(
    wvlg=wvlg,
    flux=flux,
    unc=unc,
)

2023-07-28 15:12:12,645.645 | DEBUG    | _set_displayed_data - spectrum.py:246 : Setting the following keys for displaying: ['wvlg', 'flux', 'unc']


In [10]:
spec.info()

2023-07-28 15:12:12,656.656 | INFO     | info - spectrum.py:320 : Properties of <zhunter.spectrum.OneDSpectrum object at 0x18e197ca0>:
{'flux_q025': <Quantity 0.8 erg / (Angstrom cm2 s)>,
 'flux_q975': <Quantity 1.196 erg / (Angstrom cm2 s)>,
 'flux_unit': Unit("erg / (Angstrom cm2 s)"),
 'wvlg_max': <Quantity 700. nm>,
 'wvlg_min': <Quantity 550. nm>,
 'wvlg_span': <Quantity 150. nm>,
 'wvlg_step': <Quantity [0.02, 0.02, 0.02, ..., 0.02, 0.02, 0.02] nm>,
 'wvlg_unit': Unit("nm")}
Displayed properties of <zhunter.spectrum.OneDSpectrum object at 0x18e197ca0>:
{'flux_q025_disp': <Quantity 0.8 erg / (Angstrom cm2 s)>,
 'flux_q975_disp': <Quantity 1.196 erg / (Angstrom cm2 s)>,
 'wvlg_max_disp': <Quantity 700. nm>,
 'wvlg_min_disp': <Quantity 550. nm>,
 'wvlg_span_disp': <Quantity 150. nm>,
 'wvlg_step_disp': <Quantity [0.02, 0.02, 0.02, ..., 0.02, 0.02, 0.02] nm>}


## Change the units
In case the spectrum was read with wrong units

In [11]:
spec.set_flux_unit(u.Unit('1e-18 erg/s/cm2/AA'))
# spec.set_wvlg_unit(u.Unit('nm'))

2023-07-28 15:12:17,055.055 | WARNING  | _showwarning - logger.py:237 : UnitsWarning: '1e-18 erg/s/cm2/AA' contains multiple slashes, which is discouraged by the FITS standard


## Visualizing the spectrum

In [12]:
# Classic plotting widget
# fig = pg.PlotWidget()
# vb = fig.plotItem.vb

# or custom plotting widget
from zhunter.OneDGraphicsWidget import OneDGraphicsWidget
fig = OneDGraphicsWidget()
fig.set_up_plot()
vb = fig.ax1D.vb

fig.show()

2023-07-28 15:12:17,888.888 | INFO     | set_up_plot - OneDGraphicsWidget.py:72 : Setting up a new plot called '1D'


In [13]:
vb.addItem(spec.PlotItem)
vb.addItem(spec.PlotItem_unc)

In [14]:
spec.update_plotted_items()

2023-07-28 15:12:20,301.301 | INFO     | update_plotted_items - spectrum.py:195 : Updating displayed spectrum
2023-07-28 15:12:20,302.302 | DEBUG    | update_plotted_items - spectrum.py:209 : No bounds provided


In [ ]:
# Add another spectrum
spec2 = OneDSpectrum(color=colors['specsys'][2])

In [24]:
wvlg, flux, unc = generate_fake_1D_spectrum(
    flux_scale=1,
    SNR=20,
    emission_line={'amplitude':10},
)

spec2.load_from_data(
    wvlg=wvlg,
    flux=flux,
    unc=unc,
)
vb.addItem(spec2.PlotItem)
vb.addItem(spec2.PlotItem_unc)
spec2.update_plotted_items(bounds=(640*u.nm, 680*u.nm))

2023-07-28 15:20:24,066.066 | DEBUG    | generate_fake_1D_spectrum - misc.py:87 : Adding an emission line with mean: 656.28, stddev: 1, amplitude: 10
2023-07-28 15:20:24,069.069 | DEBUG    | _set_displayed_data - spectrum.py:246 : Setting the following keys for displaying: ['wvlg', 'flux', 'unc']
2023-07-28 15:20:24,074.074 | INFO     | update_plotted_items - spectrum.py:195 : Updating displayed spectrum
2023-07-28 15:20:24,075.075 | DEBUG    | update_plotted_items - spectrum.py:198 : Bounds provided: (<Quantity 640. nm>, <Quantity 680. nm>)


## Rebinning

In [20]:
spec.convolve_gaussian(sigma=3)

2023-07-28 15:15:16,066.066 | WARNING  | convolve_gaussian - spectrum.py:336 : Error/uncertainty propagation is not implemented for convolution.
2023-07-28 15:15:16,067.067 | DEBUG    | _set_displayed_data - spectrum.py:246 : Setting the following keys for displaying: ['flux']
2023-07-28 15:15:31,498.498 | DEBUG    | keyPressEvent - OneDGraphicsWidget.py:283 : Key: Control, Mouse position: [387,252]


In [21]:
spec.update_plotted_items()

2023-07-28 15:15:32,887.887 | INFO     | update_plotted_items - spectrum.py:195 : Updating displayed spectrum
2023-07-28 15:15:32,887.887 | DEBUG    | update_plotted_items - spectrum.py:209 : No bounds provided
2023-07-28 15:15:41,813.813 | DEBUG    | keyPressEvent - OneDGraphicsWidget.py:283 : Key: Control, Mouse position: [350,214]


In [22]:
spec.info()

2023-07-28 15:15:44,899.899 | INFO     | info - spectrum.py:320 : Properties of <zhunter.spectrum.OneDSpectrum object at 0x18e197ca0>:
{'flux_q025': <Quantity 0.8 1e-18 erg / (Angstrom cm2 s)>,
 'flux_q975': <Quantity 1.196 1e-18 erg / (Angstrom cm2 s)>,
 'flux_unit': Unit("1e-18 erg / (Angstrom cm2 s)"),
 'wvlg_max': <Quantity 700. nm>,
 'wvlg_min': <Quantity 550. nm>,
 'wvlg_span': <Quantity 150. nm>,
 'wvlg_step': <Quantity [0.02, 0.02, 0.02, ..., 0.02, 0.02, 0.02] nm>,
 'wvlg_unit': Unit("nm")}
Displayed properties of <zhunter.spectrum.OneDSpectrum object at 0x18e197ca0>:
{'flux_q025_disp': <Quantity 0.939 1e-18 erg / (Angstrom cm2 s)>,
 'flux_q975_disp': <Quantity 1.059 1e-18 erg / (Angstrom cm2 s)>,
 'flux_unit': Unit("1e-18 erg / (Angstrom cm2 s)"),
 'gauss_convolution_sigma': 3,
 'wvlg_max_disp': <Quantity 700. nm>,
 'wvlg_min_disp': <Quantity 550. nm>,
 'wvlg_span_disp': <Quantity 150. nm>,
 'wvlg_step_disp': <Quantity [0.02, 0.02, 0.02, ..., 0.02, 0.02, 0.02] nm>,
 'wvlg_unit

In [26]:
spec2.PlotItem.clear()

In [38]:
from pathlib import Path

In [43]:
from itertools import cycle
type(cycle([1,2,3]))

itertools.cycle